In [1]:
import pypsa
import matplotlib.pyplot as plt
import pandas as pd
nBase = pypsa.Network("../../results/smart/current/networks/base_s_27_3H_3H_2045.nc")

# pathTail = "/networks/base_s_27_3H_3H_2045.nc"
# nBase = pypsa.Network(f"../../results/scenario/base{pathTail}")

%load_ext autoreload
%autoreload 2

from pea import Pea, carriers as cs

pea = Pea(nBase, config={
  'resolution': 3
})

INFO:pypsa.io:Imported network base_s_27_3H_3H_2045.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [9]:
need = {
  'electrolysis': pea.get(cs.h2Electrolysis).energy('p0') / 1e6,
  'power2warm': pea.get(cs.power2warm).energy('p0') / 1e6,
  'bev': pea.get(cs.landTransportEV).energy('p') / 1e6,
  'electricity': pea.get(cs.powerUse).energy('p') /1e6,
  'inudstrz': pea.get(cs.powerUseIndustry).energy('p') /1e6,
}

totalNeed = 0
for key,value in need.items():
  print(key, value)
  totalNeed = totalNeed + value

export = pea.get(cs.power, type='import').t('p1').clip(lower=0).sum(axis=1) + pea.get(cs.power, type='export').t('p0').clip(lower=0).sum(axis=1)
pimport = - pea.get(cs.power, type='import').t('p1').clip(upper=0).sum(axis=1) - pea.get(cs.power, type='export').t('p0').clip(upper=0).sum(axis=1)

print('export', export.sum() * 3 / 1e6)
print('import', pimport.sum() * 3 /1e6)
print('netImport', export.sum() * 3 / 1e6 - pimport.sum() * 3 /1e6)
print('warmLoad', pea.get(cs.warmLoad).energy('p') / 1e6)
print('totalNeed', totalNeed)


838

electrolysis 715.3688622692058
power2warm 207.06909364740557
bev 141.3790481337635
electricity 206.0006891037039
inudstrz 354.2019657533789
export 154.70827300353122
import 169.354104233931
netImport -14.645831230399779
warmLoad 557.7357981830058
totalNeed 1624.0196589074576


838

In [14]:
df = pd.DataFrame(index = nBase.snapshots)

for carrier in ['onwind', 'offwind', 'solar', 'solarRooftop']:
  df[carrier] = pea.get(getattr(cs,carrier)).t('p').sum(axis=1)

for carrier in ['batteryDischarger', 'gasH2Power', 'biomassCHP', 'geothermalORC', 'v2g', 'h2FC']:
  carrier_t = pea.get(getattr(cs,carrier)).t('p', pAs='p1')
  if carrier_t is not None:
    df[carrier] = - carrier_t.sum(axis=1)
df['phs'] = pea.get(cs.phs).t('p_dispatch').sum(axis=1)
df['importPower'] = - pea.get(cs.power, type='import').t('p1').clip(upper=0).sum(axis=1) \
                    - pea.get(cs.power, type='export').t('p0').clip(upper=0).sum(axis=1)

totalSupply = df.sum().sum() * 3/ 1e6


# print('netImport', totalNeed - totalSupply)
print(totalSupply - totalNeed)

print(df.sum() *3/ 1e6)

-476.5662650741274
onwind               361.327258
offwind               32.120855
solar                319.507566
solarRooftop         156.760050
batteryDischarger     64.130692
gasH2Power             0.369980
biomassCHP             1.204068
geothermalORC          0.000013
v2g                   39.668789
h2FC                   0.000010
phs                    3.010009
importPower          169.354104
dtype: float64


In [19]:
up = - pea.get(cs.power, type='inner').t('p0').clip(upper=0).sum(axis=1).sum()
low = pea.get(cs.power, type='inner').t('p0').clip(lower=0).sum(axis=1).sum()


total = up + low 
total.sum() * 3 /1e6


pea.get(cs.onwind).t('p').sum(axis=1)

snapshot
2019-01-01 00:00:00     82396.259717
2019-01-01 03:00:00    107958.973464
2019-01-01 06:00:00    124861.054989
2019-01-01 09:00:00    124543.204747
2019-01-01 12:00:00    133105.768390
                           ...      
2019-12-31 09:00:00     48622.788548
2019-12-31 12:00:00     36107.228034
2019-12-31 15:00:00     29175.372339
2019-12-31 18:00:00     25083.488656
2019-12-31 21:00:00     17656.266026
Length: 2920, dtype: float64

In [ ]:
df = pd.Series()

for carrier in ['onwind', 'offwind', 'solar', 'solarRooftop']:
  df[carrier] = pea.get(getattr(cs,carrier)).df['p_nom_opt'].sum() 

for carrier in ['batteryDischarger', 'gasH2Power', 'biomassCHP', 'geothermalORC', 'v2g', 'h2FC']:
  df[carrier] = pea.get(getattr(cs,carrier)).df['p_nom_opt'].sum() 

#   if carrier_t is not None:
#     df[carrier] = - carrier_t.sum(axis=1)
# df['phs'] = pea.get(cs.phs).t('p_dispatch').sum(axis=1)
# df['importPower'] = - pea.get(cs.power, type='import').t('p1').clip(upper=0).sum(axis=1) \
#                     - pea.get(cs.power, type='export').t('p0').clip(upper=0).sum(axis=1)

# totalSupply = df.sum().sum() * 3/ 1e6


# print('netImport', totalNeed - totalSupply)
# print(totalSupply - totalNeed)

print(df)

onwind          180000.000115
offwind           6434.768970
solar           336000.056101
solarRooftop    164000.000307
dtype: float64


In [23]:
pea.get(cs.onwind).df['p_nom_opt'].sum()

180000.0001148726